In [1]:
import numpy as np
from numpy.random import choice



for reference: http://mamallamacoding.blogspot.com/2014/03/making-tictactoe-with-object-oriented.html

### Outline:
- Game  
-- instantiates new Players, or accesses provided Players  
-- assigns marks to Players  
-- instantiates the Board  
-- keeps track of whose turn it is  
-- passes Board to Players when asking for a move, then updates Board with move  
-- keeps track of game outcome
- Board  
-- returns totals for each Player
- Player  
-- can exist on its own outside of Game  
-- has a policy  
-- selects a move  
-- remembers Board state from Player's previous move, and updates policy if needed  

# Classes

In [ ]:
class ttt_board(object):
    
    def __init__(self):
        self.spaces = ['_'] * 9
        
    def __str__(self):
        # Allow for printing of board in human-readable format
        return '%s\n%s\n%s\n' % (' '.join(self.spaces[:3]), ' '.join(self.spaces[3:6]), ' '.join(self.spaces[6:]))
    
    def update(self, ix, mark):
        self.spaces[ix] = mark

    
class ttt_player(object):
    
    def __init__(self, player_type, mark, epsilon=.2, alpha=.1):
        self.player_type = player_type
        self.mark = mark
        self.board_vals = [2**i for i in range(9)]
        if player_type == 'computer':
            self.policy = dict()
            self.alpha = alpha            # learning rate
            self.epsilon = epsilon        # exploration rate
            self.prev_state = None
    
    
    def __str__(self):
        return 'Player %s' % self.mark
    
    
    def set_epsilon(self, epsilon):
        self.epsilon = epsilon
    
    
    def get_player_total(self, state):
        total = 0
        for i, x in enumerate(state):
            if x == self.mark:
                total += self.board_vals[i]
        return total
    
    
    def get_move_outcome(self, state):
        win_totals = [7, 56, 448, 73, 146, 292, 273, 84]
        total = self.get_player_total(state)
        for val in win_totals:
            if total & val == val: # bitwise operation!
                return 'win'
        if '_' not in state:
            return 'draw'
        return None
    
    
    def get_prob_from_policy(self, state):
        state = tuple(state)
        if state in self.policy:
            return self.policy[state]
        
        outcome = self.get_move_outcome(state)
        if outcome == 'win':
            self.policy[state] = 1
        elif outcome == 'draw':
            self.policy[state] = .5
        else:                       # Fill with .5 if winner hasn't been decided yet
            self.policy[state] = .5
        return self.policy[state]

                                   
    def update_policy(self, current_win_prob, board, state):
        """
        Updates the win probability of a previous state based on the win probability of the current state.
        """
        prev_state = tuple(self.prev_state)
        prev_win_prob = self.policy[prev_state]
        self.policy[prev_state] = prev_win_prob + self.alpha*(current_win_prob - prev_win_prob)
        
        
    def get_move(self, board):
        if self.player_type == 'human':
            while True:
                try:
                    row_index, col_index = eval(raw_input( \
                                "Enter the row and column you want to make a mark in, using the format x,y "))
                    # Get correct index in the board list, adjusting for 0-indexing
                    move_index = (row_index-1)*3 + col_index - 1
                    if board.spaces[move_index] == '_':
                        return move_index
                except:
                    pass
                print "Invalid move.\n"
        
        # Make Computer player decision
        state = board.spaces[:]
        possible_move_indices = [i for i, x in enumerate(state) if x == '_']
        
        if np.random.uniform() < self.epsilon:     # randomly choose a move with p = epsilon
            move_index = choice(possible_move_indices)
            next_state = state[:]
            next_state[move_index] = self.mark
            _ = self.get_prob_from_policy(next_state) # I don't use the result, but this function fills the
                            # policy with a win probability for this state if this state key doesn't already exist
        
        else:                                     # Choose the best known move with p = 1-epsilon
            max_prob = 0
            best_move_ixs = []
            for i in possible_move_indices:
                potential_state = state[:]
                potential_state[i] = self.mark
                prob = self.get_prob_from_policy(potential_state)
                if prob == max_prob:
                        best_move_ixs.append(i)
                if prob > max_prob:
                        best_move_ixs = [i]
                        max_prob = prob
            move_index = choice(best_move_ixs)
            if self.prev_state:
                self.update_policy(max_prob, board, state)  

        self.prev_state = state
        self.prev_state[move_index] = self.mark
        return move_index
                            
            
# Game object instantiates the board and human or CPU players
class ttt_game(object):
    
    def __init__(self, playerX, playerO, print_output = True):
        self.board = ttt_board()
        if isinstance(playerX, ttt_player):
            if playerX.mark == 'X':
                self.playerX = playerX
                self.playerX.prev_state = None
            else:
                raise ValueError('playerX must have player mark "X".')
        elif playerX == 'human':
            self.playerX = ttt_player('human', 'X')
        elif playerX == 'computer':
            self.playerX = ttt_player('computer', 'X')
        else:
            raise ValueError('playerX must be a ttt_player object, or it must be a string indicating "human" or "computer."')
          
        if isinstance(playerO, ttt_player):
            if playerO.mark == 'O':
                self.playerO = playerO
                self.playerO.prev_state = None
            else:
                raise ValueError('playerO must have player mark "O".')
        elif playerO == 'human':
            self.playerO = ttt_player('human', 'O')
        elif playerO == 'computer':
            self.playerO = ttt_player('computer', 'O')
        else:
            raise ValueError('playerO must be a ttt_player object, or it must be a string indicating "human" or "computer."')
               
        self.current_player = self.playerX
        self.print_output = print_output
        
        
    def set_print(self, print_output):
        """
        print_output: True or False
        """
        self.print_output = print_output
    
    
    def take_a_turn(self):
        if self.print_output:
            print "%s's turn:" % self.current_player
        move_index = self.current_player.get_move(self.board)
        self.board.update(move_index, self.current_player.mark)
        if self.current_player == self.playerX:
            self.current_player = self.playerO
        else:
            self.current_player = self.playerX
        if self.print_output:
            print self.board
        
        
    def game_outcome(self):
        state = self.board.spaces
        win_totals = [7, 56, 448, 73, 146, 292, 273, 84]
        X_total = self.playerX.get_player_total(state)
        O_total = self.playerO.get_player_total(state)
        for val in win_totals:
            if X_total & val == val:        # Bitwise operation!
                return 'Player X wins!'
            if O_total & val == val:
                return 'Player O wins!'
        if '_' not in state:
            return 'The game is a tie!'
        return None

                                   
    def send_game_end_signal(self, outcome):
        """
        The "current player" is the player who took the penultimate turn. We send that player information on
        the game outcome, so that player can adjust its policy.
        """
        penultimate_player = self.current_player
        if outcome == 'The game is a tie!':
            win_prob = .5
        else:
            win_prob = 0
        if penultimate_player.player_type == 'computer':
            penultimate_player.update_policy(win_prob, self.board, self.board)   
    
    
    def play(self):
        if self.print_output:
            print self.board
        outcome = self.game_outcome()
        while not (outcome):
            self.take_a_turn()
            outcome = self.game_outcome()
        self.send_game_end_signal(outcome)
        if self.print_output:
            print outcome
      
    
    def reset(self):
        self.board = ttt_board()
        self.playerX.prev_state = None
        self.playerO.prev_state = None
        self.current_player = self.playerX

 

# Helper functions

In [38]:
def print_board(state):
    """
    state: list of strings that indicates the board state
    """
    print '%s\n%s\n%s' % (' '.join(state[:3]), ' '.join(state[3:6]), ' '.join(state[6:]))
                          

def train_computer_players(player1, player2, n_games=200000):
    player1.set_epsilon(.9)
    player2.set_epsilon(.9)
    game = ttt_game(player1, player2, print_output = False)
    for i in range(n_games):
        game.play()
        game.reset()
        

def test_computer(computer_player, n_games=3):
    """
    Plays n_games with a human versus the computer_player
    """    
    computer_player.set_epsilon(0)
    if computer_player.mark == 'X':
        game = ttt_game(computer_player, 'human', print_output=True)
    if computer_player.mark == 'O':
        game = ttt_game('human', computer_player, print_output=True)
    for i in range(n_games):
        game.play()
        game.reset()
        
        
def explore_policy(state, player):
    print_board(state)
    print 'Initial board state\n'
    possible_move_indices = [i for i, x in enumerate(state) if x == '_']
    for i in possible_move_indices:
        potential_state = state[:]
        potential_state[i] = player.mark        
        print_board(potential_state)
        print 'Win probability = %.17f\n' % player.policy[tuple(potential_state)]
    

In [30]:
player1 = ttt_player('computer', 'X', epsilon=.9)
player2 = ttt_player('computer', 'O', epsilon=.9)

In [31]:
train_computer_players(player1, player2)

In [40]:
state = ['X', '_', '_', '_', '_', '_', '_', '_', '_']
explore_policy(state, player2)

X _ _
_ _ _
_ _ _
Initial board state

X O _
_ _ _
_ _ _
Win probability = 0.69160552030801892

X _ O
_ _ _
_ _ _
Win probability = 0.76181671711009102

X _ _
O _ _
_ _ _
Win probability = 0.62568056720914855

X _ _
_ O _
_ _ _
Win probability = 0.91438622016708337

X _ _
_ _ O
_ _ _
Win probability = 0.77919632249309156

X _ _
_ _ _
O _ _
Win probability = 0.79848202634048260

X _ _
_ _ _
_ O _
Win probability = 0.75209301436524290

X _ _
_ _ _
_ _ O
Win probability = 0.73044782152507515



In [41]:
test_computer(player2)

_ _ _
_ _ _
_ _ _

Player X's turn:
Enter the row and column you want to make a mark in, using the format x,y 1,1
X _ _
_ _ _
_ _ _

Player O's turn:
X _ _
_ O _
_ _ _

Player X's turn:
Enter the row and column you want to make a mark in, using the format x,y 3,3
X _ _
_ O _
_ _ X

Player O's turn:
X O _
_ O _
_ _ X

Player X's turn:
Enter the row and column you want to make a mark in, using the format x,y 3,2
X O _
_ O _
_ X X

Player O's turn:
X O _
_ O _
O X X

Player X's turn:
Enter the row and column you want to make a mark in, using the format x,y 1,3
X O X
_ O _
O X X

Player O's turn:
X O X
_ O O
O X X

Player X's turn:
Enter the row and column you want to make a mark in, using the format x,y 2,1
X O X
X O O
O X X

The game is a tie!
_ _ _
_ _ _
_ _ _

Player X's turn:
Enter the row and column you want to make a mark in, using the format x,y 2,2
_ _ _
_ X _
_ _ _

Player O's turn:
_ _ _
_ X _
O _ _

Player X's turn:
Enter the row and column you want to make a mark in, using the

# Mental Bookmark
- update documentation (say what each class object does)
- create a module that I can import?
- try looking at various statistics as my players learn stuff?
- give computer players one policy dictionary to share?  
-- they'll train twice as fast, and have all entries